In [0]:
%pip install delta-spark

Python interpreter will be restarted.
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812390 sha256=577f1a9c8ee7c692e247444aab0685a591eb0cf278e09fff9bbc086d725c1f4d
  Stored in directory: /root/.cache/pip/wheels/11/67/ea/33c283e520b775aa7a7a0d404447e287be841a711d074d4d91
Successfully built pyspark
Python interpreter will be restarted.


In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("BronzeLayerIngestion").getOrCreate()

In [0]:
df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/asmita9andraskar@gmail.com/Product_Review.csv",inferScheme=True)

In [0]:
df.show()

+---------+------+------------+----------------+--------------------+------+--------------------+
|ProductID|UserID|    UserName|      ReviewDate|           UserEmail|Rating|          ReviewText|
+---------+------+------------+----------------+--------------------+------+--------------------+
|      709|     1|  John Smith|18-09-2013 00:00|john@fourthcoffee...|     5|I can't believe I...|
|      937|     2|       David|13-11-2013 00:00|david@graphicdesi...|     4|A little on the h...|
|      937|     3|        Jill|15-11-2013 00:00|jill@margiestrave...|     2|Maybe it's just b...|
|      798|     4|Laura Norman|15-11-2013 00:00|laura@treyresearc...|     5|The Road-550-W fr...|
+---------+------+------------+----------------+--------------------+------+--------------------+



In [0]:
# Select and display only specific columns from the DataFrame
df.select("ReviewText", "Rating", "ProductID").show()


+--------------------+------+---------+
|          ReviewText|Rating|ProductID|
+--------------------+------+---------+
|I can't believe I...|     5|      709|
|A little on the h...|     4|      937|
|Maybe it's just b...|     2|      937|
|The Road-550-W fr...|     5|      798|
+--------------------+------+---------+



In [0]:
dbutils.fs.mkdirs("/mnt/bronze/layer/delta_table")


Out[6]: True

In [0]:
df.write.format("delta").mode("overwrite").save("/mnt/bronze/layer/delta_table")

In [0]:
dbutils.fs.ls('dbfs:/mnt/bronze/layer/delta_table/')


Out[8]: [FileInfo(path='dbfs:/mnt/bronze/layer/delta_table/_delta_log/', name='_delta_log/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/layer/delta_table/part-00000-0b21fd5d-cb7c-4237-839b-adca67fd4f9d-c000.snappy.parquet', name='part-00000-0b21fd5d-cb7c-4237-839b-adca67fd4f9d-c000.snappy.parquet', size=9094, modificationTime=1721992059000),
 FileInfo(path='dbfs:/mnt/bronze/layer/delta_table/part-00000-277743b1-cae5-41db-bccd-2d643407f474-c000.snappy.parquet', name='part-00000-277743b1-cae5-41db-bccd-2d643407f474-c000.snappy.parquet', size=9094, modificationTime=1724235703000),
 FileInfo(path='dbfs:/mnt/bronze/layer/delta_table/part-00000-9c783b94-595c-48fc-85d5-dec2fb9ff1ab-c000.snappy.parquet', name='part-00000-9c783b94-595c-48fc-85d5-dec2fb9ff1ab-c000.snappy.parquet', size=9094, modificationTime=1721992170000)]

In [0]:
df_bronze = spark.read.format("delta").load("/mnt/bronze/layer/delta_table/")

In [0]:
df_bronze.show()

+---------+------+------------+----------------+--------------------+------+--------------------+
|ProductID|UserID|    UserName|      ReviewDate|           UserEmail|Rating|          ReviewText|
+---------+------+------------+----------------+--------------------+------+--------------------+
|      709|     1|  John Smith|18-09-2013 00:00|john@fourthcoffee...|     5|I can't believe I...|
|      937|     2|       David|13-11-2013 00:00|david@graphicdesi...|     4|A little on the h...|
|      937|     3|        Jill|15-11-2013 00:00|jill@margiestrave...|     2|Maybe it's just b...|
|      798|     4|Laura Norman|15-11-2013 00:00|laura@treyresearc...|     5|The Road-550-W fr...|
+---------+------+------------+----------------+--------------------+------+--------------------+



In [0]:
df_bronze.printSchema()

root
 |-- ProductID: string (nullable = true)
 |-- UserID: string (nullable = true)
 |-- UserName: string (nullable = true)
 |-- ReviewDate: string (nullable = true)
 |-- UserEmail: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- ReviewText: string (nullable = true)



In [0]:
df_bronze.select("ReviewText", "Rating", "ProductId").show()


+--------------------+------+---------+
|          ReviewText|Rating|ProductId|
+--------------------+------+---------+
|I can't believe I...|     5|      709|
|A little on the h...|     4|      937|
|Maybe it's just b...|     2|      937|
|The Road-550-W fr...|     5|      798|
+--------------------+------+---------+



In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("example").getOrCreate()

df = spark.read.format("csv").option("header","true").option("inferschema","true").load("/FileStore/tables/olympics2024.csv")

df.show()

df.printSchema()

df.write.mode("overwrite").format("parquet").save("/FileStore/tables/new_data")

# Original columns: ["Rank", "Country", "Country Code", "Gold", "Silver", "Bronze", "Total"]

# Clean column names
new_columns = [col.replace(' ', '_').replace('|', '') for col in df.columns]
df = df.toDF(*new_columns)

# Write to Delta format
df.write.mode("overwrite").format("delta").save("/FileStore/tables/new_data1")